<a href="https://colab.research.google.com/github/ar040701/DeepLearningProjects/blob/main/CIFAR_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import torch.nn.functional as f
import torch.optim as optim
from torch.utils.data import DataLoader

In [56]:
devices=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
devices

device(type='cuda')

In [57]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_dataset=datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
test_dataset=datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)

In [58]:
#Getting features and labels from the first image
features,labels=train_dataset[0]
features.shape,labels

(torch.Size([3, 32, 32]), 6)

In [59]:
train_loader=DataLoader(train_dataset,batch_size=32, shuffle=True,num_workers=2)
test_loader=DataLoader(test_dataset,batch_size=32, shuffle=False,num_workers=2)

In [60]:
classes_name=['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']

In [61]:
#Define a model
class CNN(nn.Module):
    def __init__(self):
      super().__init__()
      self.conv1=nn.Conv2d(3,8,5) #(8,28,28)
      self.bn1=nn.BatchNorm2d(8)
      self.pool1=nn.MaxPool2d(2,2) #(8,14,14)
      self.conv2=nn.Conv2d(8,16,5)  #(16,10,10) -> flatten (16*5*5)
      self.bn2=nn.BatchNorm2d(16)
      self.dropuout1=nn.Dropout(0.25)

      #Fully connected layers
      self.ll1=nn.Linear(16*5*5, 120)
      self.ll2=nn.Linear(120,64)
      self.ll3=nn.Linear(64,10)

    def forward(self,x):
      x=f.relu(self.bn1(self.conv1(x)))
      x=self.pool1(x)
      x=f.relu(self.bn2(self.conv2(x)))
      x=self.pool1(x)
      x=self.dropuout1(x)
      x=torch.flatten(x,1)
      x=f.relu(self.ll1(x))
      x=self.ll2(x)
      return x


In [62]:
model=CNN()
loss_function=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.001, weight_decay=0.001)

In [63]:
epoch_rate=30

for epoch in range(epoch_rate):
  running_loss=0.0
  print(f'training epoch {epoch}...')
  #For training dataset
  for i,data in enumerate(train_loader):
    branch_features, branch_labels=data

    y_pred= model(branch_features)
    loss= loss_function(y_pred,branch_labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss+=loss.item()
  print(f'loss: {running_loss/len(train_loader)}')




training epoch 0...
loss: 2.474063517951233
training epoch 1...
loss: 1.7846633250028447
training epoch 2...
loss: 1.6256251885810116
training epoch 3...
loss: 1.5380605312775741
training epoch 4...
loss: 1.4791204091149572
training epoch 5...
loss: 1.4432387361904786
training epoch 6...
loss: 1.4085415297796233
training epoch 7...
loss: 1.3822915702009537
training epoch 8...
loss: 1.3585299455196675
training epoch 9...
loss: 1.3375422663209686
training epoch 10...
loss: 1.3174406777035328
training epoch 11...
loss: 1.3002672409370628
training epoch 12...
loss: 1.2792322075069522
training epoch 13...
loss: 1.2558801686313774
training epoch 14...
loss: 1.2454364381001228
training epoch 15...
loss: 1.2310979831165332
training epoch 16...
loss: 1.218834689543633
training epoch 17...
loss: 1.2082295895995654
training epoch 18...
loss: 1.1979132789644156
training epoch 19...
loss: 1.1884465900202708
training epoch 20...
loss: 1.1842118250355076
training epoch 21...
loss: 1.1757524776214678


In [64]:
torch.save(model.state_dict(),'model.pth')


In [65]:
net=CNN()
net.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [66]:
net.eval()
correct=0
total=0

with torch.no_grad():
  for data in test_loader:
    branch_features, branch_labels=data
    outputs=net(branch_features)
    _,predicted=torch.max(outputs,1) #returns a tuple (value, index)
    total+=branch_labels.size(0)
    correct+=(predicted==branch_labels).sum().item()
accuracy=100*correct/total
print(f'Accuracy of the network on the 10000 test images: {accuracy}%')


Accuracy of the network on the 10000 test images: 61.44%


In [72]:
from PIL import Image
new_transform=transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

def load_image(image_path):
  image=Image.open(image_path)
  image=new_transform(image)
  image=torch.unsqueeze(image,0)
  return image

image_paths = ['/download.jpg','/example2.jpg']
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
  for image in images:
    output=net(image)
    _,predicted=torch.max(output,1)
    print(f'Prediction:{classes_name[predicted]}')

Prediction:dog
Prediction:horse
